# Index Swapping Assignment
### BI 622
### Zach Goode

---

## Question 1


### Average quality scores by position

The script below was run as follows on the HPC:

>`python ./per_bp_qual.py -f <input file> -o <output ID> -l <read length>`

In [ ]:
### outputs average quality of each position in sequence reads

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-f',dest="file1", type=str, required=True, help='per_bp_qual.py -f1 <file_name>')
parser.add_argument('-o',dest="file_out", type=str, required=True, help='per_bp_qual.py -o <output_ID>')
parser.add_argument('-l',dest="read_len", type=str, required=True, help='per_bp_qual.py -l <read_length>')
args = parser.parse_args()

#variables which change depending on input options

file1 = args.file1 #specified by -f
file_naming = args.file_out #specified by -o
read_len = args.read_len #specified by -l

#Empty array with length equal to length of reads
mean_scores=[]
i=0
while i < int(read_len):
    i+=1
    mean_scores.append(0.0)

with open(file1, "r") as file:
    i=0 #counter for modulus
    NR=0 #line counter
    for line in file:
        i+=1
        if i%4 == 0:
            NR+=1
            for bp in range(int(read_len)):
                mean_scores[bp] = mean_scores[bp] + (ord(line[bp])-33)

#create output file with unique name (e.g. "R1_per_bp.tsv")
outfile1=open(file_naming+"_per_bp.tsv",'w')
#header line
outfile1.write("bp position\tmean quality score\n")

#extract and format quality scores into tsv output
for pos in range(len(mean_scores)):
    bp=pos+1
    mean_pos=mean_scores[pos]/NR
    outfile1.write(str(bp)+"\t"+str(mean_pos)+"\n")
            
outfile1.close()           

# 1. a.

# Distributions of positional quality scores

(Plotted with R)

![test](./per_bp.png)

---

## Average quality scores per read

The script below was run as followson the HPC:

>`python ./per_read_qual.py -f <input file> -o <output ID> -l <read length>`


In [ ]:
#!/usr/bin/env python

### outputs average quality per read

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-f',dest="file1", type=str, required=True, help='per_bp_qual.py -f1 <file_name>')
parser.add_argument('-o',dest="file_out", type=str, required=True, help='per_bp_qual.py -o <output_ID>')
parser.add_argument('-l',dest="read_len", type=str, required=True, help='per_bp_qual.py -l <read_length>')
args = parser.parse_args()

#parameters which change depending on input file
file1 = args.file1
file_naming=args.file_out
read_len=args.read_len

#array holding average quality for each read
mean_read_score=[]

with open(file1, "r") as file:
    i=0 # for modulus
    NR=0 # count number of reads
    for line in file:
        i+=1
        if i%4 == 0:
            NR+=1
            current_read=0
            for bp in range(int(read_len)):
                current_read = current_read + (ord(line[bp])-33)
            mean_read_score.append(current_read)
            
for item in range(0,len(mean_read_score)):
    mean_read_score[item] = mean_read_score[item]/int(read_len)
            
# create output file with unique name (e.g. "R1_per_read.tsv")          
outfile1=open(file_naming+"_per_read.tsv",'w')
# header line
outfile1.write("mean quality score of read\n")

for val in mean_read_score:
    outfile1.write(str(val)+"\n")
            
outfile1.close()

# Distributions of quality scores per read

## Sequence reads:

![per_read_sequence](./per_read_seq.png)

## Index reads:

![per_read_index](./per_read_index.png)

## The data for the histograms above were binned using the following code:

In [ ]:
### bins data using numpy.histogram and outputs to tsv

parser = argparse.ArgumentParser()
parser.add_argument('-f',dest="file1", type=str, required=True, help='per_bp_qual.py -f1 <file_name>')
parser.add_argument('-o',dest="file_naming", type=str, required=True, help='per_bp_qual.py -o <name_prefix>')
args = parser.parse_args()

#input options (-f and -o)
file=args.file1
file_naming = args.file_naming #used as a prefix for naming output file

mean_per_read=[]

with open(file, 'r') as fh:
    i=0
    for line in fh:
        if i > 0:
            mean_per_read.append(float(line))
        i+=1

data = np.asarray(mean_per_read)
bin_means = np.histogram(data, bins=20, range=(21,41))

outfile=open(file_naming+"_hist.tsv",'w')

for j in range(len(bin_means[0])):
    outfile.write(str(bin_means[0][j])+"\t"+str(int(bin_means[1][j]))+"\n")

outfile.close()

---

# 1. b.

### A good cutoff for quality score appears to be around 31. This should remove most of the erroneous index pairs, while retaining the desired pairs.

# 1. c.

### Number of reads containings Ns in the index files:

>`cat 1294_S1_L008_R2_001.fastq | awk ' NR % 4 == 2 ' | grep 'N' | wc -l`
>
### 3,976,613 index reads in the R2 file contain 1 or more Ns (0.27% of reads)
>
`cat 1294_S1_L008_R2_001.fastq | awk ' NR % 4 == 2 ' | grep 'N' | wc -l`
>
### 3,328,051 index reads in the R3 file contain 1 or more Ns (0.23% of reads)

# 1. d.

### The distribution of mean quality scores across the reads tells me that the majority of reads have an average quality of 35 or higher. The index reads, in particuluar, have lower average quality scores than the sequence reads. 

---

# Question 2

### De-multiplex the samples & document index swapping


In [ ]:
file1="test_R2.fq"
file2="test_R3.fq"

# define list of indexes used for sequencing
indexes = ["GTAGCGTA", "CGATCGAT", "GATCAAGG", "AACAGCGA", "TAGCCATG", "CGGTAATC", 
          "CTCTGGAT", "TACCGGAT", "CTAGCTCA", "CACTTCAC", "GCTACTCT", "ACGATCAG",
          "TATGGCAC", "TGTTCCGT", "GTCCTAAG", "TCGACAAG", "TCTTCGAC", "ATCATGCG",
          "ATCGTGGT", "TCGAGAGT", "TCGGATTC", "GATCTTGC", "AGAGTCCA", "AGGATAGC"]

# define function to reverse complement reverse sequence reads
def rev_comp(dna):
    dna=dna[::-1]
    reverse_comp=""
    for bp in range(len(dna)):
        if dna[bp] == "A":
            reverse_comp+="T"
        if dna[bp] == "T":
            reverse_comp+="A"
        if dna[bp] == "G":
            reverse_comp+="C"
        if dna[bp] == "C":
            reverse_comp+="G"
        if dna[bp] == "N":
            reverse_comp+="N"
    return reverse_comp

# define dict of index_pairs (key) and their counts (value)
pair_counts = {}
for z in range(len(indexes)):
    pair_counts[indexes[z]]=0
# add other categories to the dict
pair_counts["Error   "]=0
pair_counts["Index_hopped"]=0
pair_counts["Total_matched"]=0
pair_counts["Total_other"]=0


# open both files and loop through each line at the same time
with open(file1,'r') as R2:
    with open(file2,'r') as R3:
        R2_line=0 #tracks line number
        R3_line=0
        for line1 in R2:
            R2_line+=1
            line1=line1.strip()
                
            if R2_line%4 == 2:
                seq1=line1
                
            for line2 in R3:
                R3_line+=1
                if R3_line%4 != 2:
                    break
                else:
                    line2=line2.strip()
                    seq2=rev_comp(line2)
                break 
        
            # update dictionary of counts for proper matching indexes
            if R2_line%4 == 2:
                if seq1 == seq2 and seq1 in indexes:
                    pair_counts[seq1]+= 1
                    pair_counts["Total_matched"]+=1

                if seq1 != seq2:
                    if seq1 and seq2 in indexes:
                        pair_counts["Index_hopped"]+=1
                        pair_counts["Total_other"]+=1
                    else:
                        pair_counts["Error   "]+=1
                        pair_counts["Total_other"]+=1

pair_counts["Total    "] = pair_counts["Total_matched"]+pair_counts["Total_other"]
                    
# format output for standard out
print("--"*23)
print("Barcode","Retained_pairs", "%",sep="\t")
i=0
for key in pair_counts:
    i+=1
    percent = 100*float(pair_counts[key]/pair_counts["Total    "])
    print(key,pair_counts[key],round(percent,3),sep="\t")
    if i == len(indexes):
          print("---------SUMMARY---------")
print("--"*23)

## Output from HPC:

# Graphical distribution:

![all_indexes](./demulti.png)

---

![summary](./demulti_summ.png)
>\**Note: Error refers to reads where indexes did not match, and at least one index was not recognized. This includes reads with Ns in the sequence.* 